In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()
df=dataset.copy()

In [ ]:
# Check and Clean data
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df.columns

In [ ]:
# Transform Categorical data to numeric
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
df['sex'] = labelencoder.fit_transform(df['sex'])
df['smoker'] = labelencoder.fit_transform(df['smoker'])
df['region'] = labelencoder.fit_transform(df['region'])

df.head()

In [ ]:
# Data visualization
df.corr()


In [ ]:
for index in df.columns[:-1]:
  sns.regplot(x=index, y="expenses", data=df)
  plt.ylim(0,)
  plt.show()
  print(df[[index, "expenses"]].corr())

In [ ]:
# Get labels
train_df = df.iloc[:,:-1]
labels_df =df['expenses']

In [ ]:
labels_df.shape

In [ ]:
labels_df.head()

In [ ]:
train_df.shape

In [ ]:
# Split data
from sklearn.model_selection import train_test_split

train_dataset,test_dataset,train_labels,test_labels = train_test_split(train_df,labels_df,test_size = 0.2,random_state = 32)


In [ ]:

train_labels.head()

In [ ]:
train_dataset.shape

In [ ]:
# Build Model
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(32, activation='relu'),
     layers.Dense(16, activation='relu'),
    layers.Dense(1)
  ])



  model.compile(loss='mse',
                optimizer=tf.keras.optimizers.RMSprop(0.001),
                metrics=['mae', 'mse'])
  return model

In [ ]:
model = build_model()
model.summary()

In [ ]:
history = model.fit(train_dataset,train_labels, validation_split=0.2, verbose=0, epochs=600)

In [ ]:
#plot the loss and validation loss of the dataset
plt.plot(history.history['mae'], label='mae')
plt.plot(history.history['val_mae'], label='val_mae')

plt.legend()

In [ ]:
scores = model.evaluate(test_dataset, test_labels, verbose = 0)

print('Mean Squared Error : ', scores[1])
print('Mean Absolute Error : ', scores[2])

In [ ]:
#rsqore evaluate
Y_pred = model.predict(test_dataset)

In [ ]:
from sklearn.metrics import r2_score

print('r2 score: ', r2_score(test_labels, Y_pred))

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
